In [893]:
%matplotlib inline  
import numpy as np 
import scipy as sp 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy.stats import norm  
sns.set_style('white') 
sns.set_context('talk')  
np.random.seed(123)

import numpy as np

import scipy.special as sp
from scipy import integrate

from sklearn import metrics

from scipy.stats import multivariate_normal

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

from sklearn import svm

from sklearn.mixture import BayesianGaussianMixture

from sklearn import naive_bayes

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif

from sklearn.feature_selection import 

from sklearn.model_selection import cross_val_score

In [2]:
def print_err(y, B):
    Penalty_1 = np.array([[-10,10], [1,0]])
    Penalty_2 = np.array([[-1,2], [1,-1]])
    
    print('AUC', metrics.roc_auc_score(y, B[1]))
    answ = np.argmax(B.T, axis=1)
    print('NUM', np.sum(answ != y))

    print('ASY1:', np.sum(Penalty_1[y, answ]))
    print('ASY2:', np.sum(Penalty_2[y, answ]))

    print('ACC:', np.sum(answ == y)/np.size(y))

In [3]:
import EM as em

In [4]:
def convert_bad_data(Data):
    Data_list = []
    for string in Data:
        Data_list.append(np.array([float(i) for i in str(string[0]).split()]))
    
    return np.array(Data_list)

In [6]:
def prob_class(X, mu, cov, w, count_of_labels = 2):
    Sum = 0
    for i in range(count_of_labels):
        Sum += w[i]*multivariate_normal.pdf(x= X, mean= mu[i], cov= cov[i])
        
    Predicted = np.array([w[i]*multivariate_normal.pdf(x = X, mean= mu[i], cov= cov[i])/Sum for i in range(count_of_labels)])
    return Predicted

Задача 1

In [991]:
np.random.seed(0)

In [992]:
X_train_load = convert_bad_data(pd.read_csv('./data/task1_1_learn_X.csv').as_matrix())
y_train_load = np.int64(np.reshape(pd.read_csv('./data/task1_1_learn_y.csv').as_matrix(), -1))

X_test_load = convert_bad_data(pd.read_csv('./data/task1_1_test_X.csv').as_matrix())

In [993]:
print(X_train_load.shape)
print(y_train_load.shape)
print(X_test_load.shape)

(999, 20)
(999,)
(999, 20)


In [988]:
NUM_K = 20
skb = SelectKBest(f_classif, k=1)
# skb = SelectKBest(mutual_info_classif, k=NUM_K)

skb.fit(X_train_load, y_train_load)


SelectKBest(k=20, score_func=<function f_classif at 0x1149f8378>)

In [989]:
np.sort(skb.scores_)[::-1][:10]

array([ 254.7839687 ,  253.06982722,  215.0965004 ,   14.36097866,
         12.74295154,   11.88314233,   10.83488993,   10.56890143,
         10.56011027,   10.55855232])

In [946]:
NUM_K = 3

In [948]:
# X = scale(np.vstack([X_train_load, X_test_load]))[:,np.where(skb.scores_ > skb.scores_[np.argsort(skb.scores_)[::-1][NUM_K]])[0]]
X = np.vstack([X_train_load, X_test_load])[:,np.where(skb.scores_ >= skb.scores_[np.argsort(skb.scores_)[::-1][NUM_K-1]])[0]]
X = (X - np.mean(X, axis=0))/np.std(X, axis=0)

X = Pruning(X, percentage_of_deleted = 0.0)
X_train = X[:X_train_load.shape[0]]
X_test = X[X_train_load.shape[0]:]


In [949]:
print(X_train.shape)
print(X_train.shape)
print(X_test.shape)

(999, 3)
(999, 3)
(9999, 3)


In [950]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train_load, test_size = 0.1)


In [951]:
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(899, 3)
(100, 3)
(899,)
(100,)


In [952]:
M = np.array([np.mean(X_train[np.where(y_train == i)[0]], axis = 0) for i in range(2)])
D = np.array([np.std(X_train[np.where(y_train == i)[0]], axis = 0) for i in range(2)])

In [953]:
print('Train')
print_err(y_train,
          prob_class(X_train, 
                     mu=M, 
                     cov=[np.diag(D[i]) for i in range(2)],
                     w = [np.sum(y_train ==i) for i in range(2)],
                     count_of_labels = 2
                    )
          )
print('Test')
print_err(y_valid,
          prob_class(X_valid, 
                     mu=M, 
                     cov=[np.diag(D[i]) for i in range(2)],
                     w = [np.sum(y_train ==i) for i in range(2)],
                     count_of_labels = 2
                    )
          )

Train
AUC 0.963399219156
NUM 96
ASY1: -4708
ASY2: -683
ACC: 0.893214682981
Test
AUC 0.962121212121
NUM 11
ASY1: -592
ASY2: -75
ACC: 0.89


In [985]:
# _count_of_clasters = 2
# _count_of_features = X_train.shape[1]
# w = np.ones(shape = [_count_of_clasters])/_count_of_clasters

# mu = np.reshape(np.random.rand(_count_of_clasters*_count_of_features), [_count_of_clasters, _count_of_features])

# # mu = np.array(M)

# cov = []
# for i in range(_count_of_clasters):
#     cov.append(np.eye(_count_of_features))
    
#     mu[i] *=-1**i
    
# cov = np.array(cov)

# w, mu, cov = em.SEM_algorithm(X_train=X_train,
#                              X_test=np.vstack([X_valid, X_test]),
#                              y_train=y_train,
#                              count_of_clasters=_count_of_clasters,
#                              w=None,
#                              mu=mu,
#                              cov=None,
#                              alpha=1.0,
#                              Num=1000
#                             )

In [980]:
# clastering_train = np.argmax(prob_class(X_train, mu=mu, cov=cov, w=w,count_of_labels=_count_of_clasters).T, axis=1)
# clastering_valid = np.argmax(prob_class(X_valid, mu=mu, cov=cov, w=w,count_of_labels=_count_of_clasters).T, axis=1)

# Data_train = []
# Answ_train = []
# Data_valid = []
# Answ_valid = []

# Models = []
# for i in range(_count_of_clasters):
#     Data_train.append(X_train[np.where(clastering_train == i)])
#     Answ_train.append(y_train[np.where(clastering_train == i)])
#     Data_valid.append(X_valid[np.where(clastering_valid == i)])
#     Answ_valid.append(y_valid[np.where(clastering_valid == i)])
    
#     Models.append(svm.SVC(probability=True, kernel='rbf'))
# #     Models.append(naive_bayes.GaussianNB())


In [983]:
# for i in range(_count_of_clasters):
#     print(Data_train[i].shape, Answ_train[i].shape, np.sum(Answ_train[i]))
#     Models[i].fit(Data_train[i], Answ_train[i])
    

In [984]:
# Answ = np.array([])
# Predict = np.array([[], []])

# for i in range(_count_of_clasters):
#     Answ = np.hstack([Answ, Answ_valid[i]])
#     if Data_valid[i].shape[0] != 0:
#         Predict = np.hstack([Predict, Models[i].predict_proba(Data_valid[i]).T])
        
# #         print(cross_val_score(estimator=Models[i], X=np.vstack([Data_train[i], Data_valid[i]]), y=np.hstack([Answ_train[i], Answ_valid[i]]), scoring='roc_auc', cv = 10), 
# #               cross_val_score(estimator=Models[i], X=np.vstack([Data_train[i], Data_valid[i]]), y=np.hstack([Answ_train[i], Answ_valid[i]]), scoring='accuracy', cv = 10))
# #         print(Models[i].predict_proba(Data_valid[i]).T)
# #         print(Answ_valid[i])
# #     print_err(Answ_valid[i], Models[i].predict_proba(Data_valid[i]).T)
# #     print('')
# Answ = np.int64(Answ)

In [ ]:
print_err(Answ, Predict)

In [976]:
def asy1(estimator, X, y):
    Penalty_1 = np.array([[-10,10], [1,0]])
    answ = np.int64(estimator.predict(X))
    return np.sum(Penalty_1[y, answ])
    
def asy2(estimator, X, y):
    Penalty_2 = np.array([[-1,2], [1,-1]])
    answ = np.int64(estimator.predict(X))
    return np.sum(Penalty_2[y, answ])

def _asy1(y1, y2):
    Penalty_1 = np.array([[-10,10], [1,0]])
    return np.sum(Penalty_1[y1, y2])

def _asy2(y1, y2):
    Penalty_2 = np.array([[-1,2], [1,-1]])
    return np.sum(Penalty_2[y1, y2])

In [977]:
print(_asy1(np.hstack([y_train, y_valid]), np.hstack([y_train, y_valid])), _asy2(np.hstack([y_train, y_valid]), np.hstack([y_train, y_valid])))



-5920 -999


In [974]:
print(cross_val_score(estimator=svm.SVC(probability=True, kernel='rbf'), X=np.vstack([X_train, X_valid]),y=np.hstack([y_train, y_valid]) , scoring='roc_auc', cv = 10)) 
print(cross_val_score(estimator=svm.SVC(probability=True, kernel='rbf'), X=np.vstack([X_train, X_valid]),y=np.hstack([y_train, y_valid]) , scoring='accuracy', cv = 10))

print(cross_val_score(estimator=svm.SVC(probability=True, kernel='rbf'), X=np.vstack([X_train, X_valid]),y=np.hstack([y_train, y_valid]) , scoring=asy1, cv = 10))
print(cross_val_score(estimator=svm.SVC(probability=True, kernel='rbf'), X=np.vstack([X_train, X_valid]),y=np.hstack([y_train, y_valid]) , scoring=asy2, cv = 10))


[ 0.94634146  0.93861789  0.98801158  0.95700703  0.96279454  0.93385697
  0.97478297  0.97245763  0.93347458  0.95084746]
[ 0.87128713  0.86138614  0.95        0.89        0.91        0.92        0.9
  0.88888889  0.85858586  0.85858586]
[-454 -472 -547 -522 -562 -544 -523 -522 -462 -500]
[-68 -67 -88 -75 -81 -82 -77 -74 -65 -67]


In [990]:
print(cross_val_score(estimator=naive_bayes.GaussianNB(), X=np.vstack([X_train, X_valid]),y=np.hstack([y_train, y_valid]) , scoring='roc_auc', cv = 10)) 
print(cross_val_score(estimator=naive_bayes.GaussianNB(), X=np.vstack([X_train, X_valid]),y=np.hstack([y_train, y_valid]) , scoring='accuracy', cv = 10))

print(cross_val_score(estimator=naive_bayes.GaussianNB(), X=np.vstack([X_train, X_valid]),y=np.hstack([y_train, y_valid]) , scoring=asy1, cv = 10)) 
print(cross_val_score(estimator=naive_bayes.GaussianNB(), X=np.vstack([X_train, X_valid]),y=np.hstack([y_train, y_valid]) , scoring=asy2, cv = 10))



[ 0.95081301  0.9495935   0.99173212  0.95328648  0.97023563  0.94584539
  0.97602315  0.97415254  0.93898305  0.96101695]
[ 0.88118812  0.86138614  0.93        0.87        0.92        0.91        0.91
  0.86868687  0.88888889  0.85858586]
[-474 -491 -545 -520 -563 -562 -562 -539 -522 -519]
[-71 -68 -84 -71 -83 -81 -81 -71 -74 -68]
